In [78]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf
import model_fsolve_v2 as modf2

import scipy.optimize as sco
import scipy.interpolate as sci

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import os
import plotly.graph_objects as go

import copy


In [79]:
import repo_config as rconfig

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter
import scienceplots
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

def to_percent(y, _):
    return f"{y * 100:.0f}\%"

In [ ]:
path = r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Inputs"
file_name = "\Garcia_Guendulain.xlsx"
hw, par, cond = fe.initialize(path, file_name)

# hw.change_riser_diameter(0.0052)
# hw.change_riser_diameter(0.0168)
# hw.change_man_diameter(0.0052)
par = hw.make_dict()

cond['rho'] = 1000
cond['eta'] = 0.001003
cond["nu"] = cond["eta"]/cond["rho"]

cond['mdot'] = 0.035 # kg/s
cond['Dv'] = (cond['mdot']/cond['rho']) # m3/s
cond['Vdot'] = cond['Dv'] * 1000 * 3600 # L/h

# cond['Vdot_Lmin'] = 480/60 # L/min
# fe.change_Vdot(cond, cond['Vdot_Lmin'] * 60)

tabl, res, PL, residuals = modf.PL_fsolve(par, cond) 
print('PL = ', res, ' Pa')

In [82]:
K_range = [1,1.5,2]
K_list = []

for k in K_range:

    hw.change_coeff(k,k,k,k)
    par = hw.make_dict()

    tabl, res, PL, residuals = modf.PL_fsolve(par, cond)

    K_list.append({'tabl' : copy.deepcopy(tabl),
                   'PL' : res,
                   'PL_tabl' : copy.deepcopy(PL),
                   'residuals' : copy.deepcopy(residuals)})

In [88]:
garcia_ref = pd.read_csv(r"G:\Drive partagés\Cercle Hard\R&D\Thèse VDE\Manuscrit\Partie II\garcia_ref_exp.csv")
garcia_ref.drop(columns=['x'], inplace=True)
garcia_ref.rename(columns={' y': 'gamma_r'}, inplace=True)
garcia_ref['qx_norm'] = garcia_ref['gamma_r'] * hw.N

garcia_numerical = pd.read_csv(r"G:\Drive partagés\Cercle Hard\R&D\Thèse VDE\Manuscrit\Partie II\garcia_ref_present_numerical.csv")
garcia_numerical.drop(columns=['x'], inplace=True)
garcia_numerical.rename(columns={' y': 'gamma_r'}, inplace=True)
garcia_numerical['qx_norm'] = garcia_numerical['gamma_r'] * hw.N

garcia_analytical = pd.read_csv(r"G:\Drive partagés\Cercle Hard\R&D\Thèse VDE\Manuscrit\Partie II\garcia_ref_present_analytical.csv")
garcia_analytical.drop(columns=['x'], inplace=True)
garcia_analytical.rename(columns={' y': 'gamma_r'}, inplace=True)
garcia_analytical['qx_norm'] = garcia_analytical['gamma_r'] * hw.N

facao_exp = pd.read_csv(r"G:\Drive partagés\Cercle Hard\R&D\Thèse VDE\Manuscrit\Partie II\facao_exp.csv")
facao_exp.drop(columns=['x'], inplace=True)
facao_exp.rename(columns={' y': 'mdotx'}, inplace=True)
facao_exp['gamma_r'] = facao_exp['mdotx'] / facao_exp['mdotx'].sum()
facao_exp['qx_norm'] = facao_exp['gamma_r'] * hw.N

facao_num = pd.read_csv(r"G:\Drive partagés\Cercle Hard\R&D\Thèse VDE\Manuscrit\Partie II\facao_corr_model.csv")
facao_num.drop(columns=['x'], inplace=True)
facao_num.rename(columns={' y': 'mdotx'}, inplace=True)
facao_num['gamma_r'] = facao_num['mdotx'] / facao_num['mdotx'].sum()
facao_num['qx_norm'] = facao_num['gamma_r'] * hw.N

In [ ]:
colors = ['olive', 'green', 'blue', 'orange', 'red', 'grey']
colors_lighter = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd', 'X', 'v', '^']

# Main plot: plotting L_f0_range on x-axis
fig, ax1 = plt.subplots(figsize=(rconfig.wcol_in, rconfig.hfig(0.3, unit="in")))

y_name = 'qx_norm'

ax1.scatter(facao_exp.index, facao_exp[y_name], label='Facao - Exp.', marker=markers[0], color = colors[5])
ax1.scatter(facao_num.index, facao_num[y_name], label='Facao - Corr. Model', marker=markers[1], color = colors[3])

# ax1.plot(garcia_ref.index, garcia_ref[y_name], 'o', label='Garcia et al. (2019) - Exp.')
ax1.scatter(garcia_analytical.index, garcia_analytical[y_name], label='Garcia - Analytical', marker=markers[3], color = colors[4])
ax1.scatter(garcia_numerical.index, garcia_numerical[y_name], label='Garcia - CFD', marker=markers[2], color = colors[0])

color_intensity = [70,100,150]

for i, k in enumerate(K_list):
    tabl = K_list[i]['tabl']
    
    tabl['gamma_r'] = tabl['qx'] / (tabl['qx'].sum())
    tabl['qx_norm'] = tabl['gamma_r'] * hw.N

    ax1.plot(tabl.index, tabl[y_name], label = f'K = {K_range[i]}', color = rconfig.get_hex_code('blue', color_intensity[i]), linestyle = linestyles[i])

# # Create the first row of the legend (for the lines)
# first_legend = fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=3, frameon=False,
#                           labels=['Model w/ MPG 20\%', 'Model w/ MPG 30\%', 'Model w/ MPG 40\%'],
#                           handles=[ax1.lines[0], ax1.lines[1], ax1.lines[2]])

# # Create the second row of the legend (for the scatter plot)
# second_legend = fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), ncol=1, frameon=False,
#                            labels=['Measure'], handles=[ax1.collections[1]])

# # Add the first legend back to the axes (to avoid being overwritten)
# ax1.add_artist(first_legend)

ax1.set_xlabel('Tube number')
ax1.set_ylabel('Normalized flow rate [-]')

ax1.set_ylim(0.92,1.12)

ax1.grid(linewidth=0.5, color=rconfig.get_hex_code('grey', 70))

fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.), ncol=2, frameon=False,)



In [ ]:
for i, k in enumerate(K_range):
    tabl = K_list[i]['tabl']

    plt.plot(tabl.index, tabl['Pin'], label = f'K = {k}')
    # plt.plot(tabl.index, tabl['Pout'], label = f'K = {k}')

plt.legend()

In [ ]:
for i, k in enumerate(K_range):
    tabl = K_list[i]['tabl']

    plt.plot(tabl.index, tabl['Pout'], label = f'K = {k}')

plt.legend()

In [111]:
garcia_analytical['own model'] = K_list[0]['tabl']['qx_norm']

In [118]:
garcia_analytical['error'] = (garcia_analytical['own model'] - garcia_analytical['qx_norm'])
garcia_analytical['relative_error'] = garcia_analytical['error'] / garcia_analytical['qx_norm']

In [ ]:
garcia_analytical

In [ ]:
mae = np.abs(garcia_analytical['error']).mean()
mae/garcia_analytical['qx_norm'].mean()

In [107]:
fig.savefig(r'G:\Mon Drive\GitHub\VDE_thesis\image\Chapter_PL\PL-2.3-Garcia-fit.png', dpi=600)

In [ ]:
D_temp = 0.0273
hw.change_man_diameter(D_temp)
hw.compute_flow(10*60, {'name':'water'}, p=1.013e5, T=25+273.15)
print('Re in manifold entrance',hw.man_in.Re)

In [ ]:
cond['Vdot_Lmin'] = 12 # L/min

D_temp = 0.0273/10
hw.change_riser_diameter(D_temp)
hw.change_man_diameter(D_temp)
par = hw.make_dict()

fe.change_Vdot(cond, cond['Vdot_Lmin'] * 60)

tabl, res, PL, residuals = modf.PL_fsolve(par, cond) 
print('PL = ', res, ' Pa')

In [61]:
eta = PropsSI('V', 'P', 1.013e5, 'T', 298.15, 'HEOS::Water')

In [67]:
hw.compute_metrics()

In [ ]:
hw.R_D

In [ ]:
res

In [63]:
hw.compute_flow(cond['Vdot'], {'name' : 'water'}, p=1.013e5, T=298.15)